In [1]:
import time
import numpy as np
import math
import itertools
from Crypto.Util import number
import Crypto.Random as Random
from Crypto.Hash import SHA256

# OUE

In [2]:
def buildParams(epsilon, width):
    d = len(CATEGORIES)
    l = decideRatio(epsilon, d, width)
    n = width
    print("n:", n, "l:", l, "d:", d)
    return d, n, l

def decideRatio(eps, d, width):
    """ Return:
            int: number of zero 
    """
    if width % 2 != 0:
        assert False, "width must be even"
    ratio = 1 / (1 + np.exp(eps))
    print('original q=', ratio)
    print('approximate q=', math.ceil(ratio * width)/100)
    return int(ratio * width)

class Prover:
    def __init__(self, data, d, n, l):
        if data in CATEGORIES:
            self.data = data
        else:
            assert False, "out of categories"
        self.d, self.n, self.l = d, n, l
        
    def setup(self):
        print('receiver setup')
        t = self.data
        print("secret input: ", t)
        vec_mu_array = [[]]*self.d
        for category in CATEGORIES:
            if category == t:
                bit_array = [0]*(self.n // 2) + [1]*(self.n // 2)
            else:
                bit_array = [0]*(self.n - self.l) + [1]*self.l
            bit_array = np.random.permutation(bit_array).tolist()
            vec_mu_array[category] = bit_array

        self.vec_mu_array = vec_mu_array
        
    def setPubKey(self, pub_key):
        self.pub_key = pub_key
    
    def step2(self, g_a_array, g_b_array, g_ab_array):
        print('step2')
        q, g, h = self.pub_key
        vec_w_array = []
        vec_y_array = []
        vec_v_array = []
        
        for mu_array, g_a, g_b, g_ab in zip(self.vec_mu_array, g_a_array, g_b_array, g_ab_array):
            w_array = []
            y_array = []
            v_array = []
            for i in range(0, self.n):
                r = number.getRandomRange(1, q-1)
                s = number.getRandomRange(1, q-1)
                w = pow(g, r, q) * pow(g_a, s, q) % q
                v = pow(g_b, r, q) * pow(g_ab * pow(g, i, q) % q, s, q) % q
                y = pow(g, mu_array[i], q) * pow(h, v, q) % q
                w_array.append(w)
                y_array.append(y)
                v_array.append(v)

            vec_w_array.append(w_array)
            vec_y_array.append(y_array)
            vec_v_array.append(v_array)

        self.vec_w_array = vec_w_array
        self.vec_y_array = vec_y_array
        self.vec_v_array = vec_v_array

        return vec_w_array, vec_y_array
    
    def AKEncBool1(self):
        print('AKEncBool1')
        q, g, h = self.pub_key
        vec_b_array = []
        vec_c_array = []
        vec_s_array = []
        vec_random_w_array = []
        for y_array, mu_array in zip(self.vec_y_array, self.vec_mu_array):
            b_array = []
            c_array = []
            s_array = []
            random_w_array = []            
            for y, mu in zip(y_array, mu_array):
                random_w = number.getRandomRange(1, q-1)
                if mu == 0:
                    c_1 = number.getRandomRange(1, q-1)
                    c_array.append(c_1)
                    s_1 = number.getRandomRange(1, q-1)
                    s_array.append(s_1)
                    b_0 = pow(h, random_w, q)
                    g_1_inv = pow(pow(g, 1, q), -1, q)
                    deno = pow(y * g_1_inv % q, c_1, q)
                    b_1 = pow(h, s_1, q) * pow(deno, -1, q) % q
                elif mu == 1:
                    c_0 = number.getRandomRange(1, q-1)
                    c_array.append(c_0)
                    s_0 = number.getRandomRange(1, q-1)
                    s_array.append(s_0)
                    b_1 = pow(h, random_w, q)
                    g_0_inv = pow(pow(g, 0, q), -1, q)
                    deno = pow(y * g_0_inv % q, c_0, q)
                    b_0 = pow(h, s_0, q) * pow(deno, -1, q) % q
                else:
                    assert False, "mu is invalid (0 or 1)"

                b_array.append((b_0, b_1))
                random_w_array.append(random_w)

            vec_b_array.append(b_array)
            vec_c_array.append(c_array)
            vec_s_array.append(s_array)
            vec_random_w_array.append(random_w_array)

        self.vec_b_array = vec_b_array
        self.vec_c_array = vec_c_array
        self.vec_s_array = vec_s_array
        self.vec_random_w_array = vec_random_w_array
        
        return vec_b_array
    
    def AKEncBool3(self, vec_ak_enc_bool_x_array):
        print('AKEncBool3')
        q, g, h = self.pub_key
        vec_c_array = []
        vec_s_array = []
        for ak_enc_bool_x_array, c_array, s_array, mu_array , v_array , random_w_array in zip(
            vec_ak_enc_bool_x_array,
            self.vec_c_array,
            self.vec_s_array,
            self.vec_mu_array,
            self.vec_v_array,
            self.vec_random_w_array
        ):
            c_array = []
            s_array = []
            for x,c,s,mu,v,random_w in zip(
                ak_enc_bool_x_array,
                c_array,
                s_array,
                mu_array,
                v_array,
                random_w_array
            ):
                if mu == 0:
                    c_0 = (x - c)
                    c_1 = c
                    s_0 = ((v * c_0) + random_w)
                    s_1 = s
                elif mu == 1:
                    c_0 = c
                    c_1 = (x - c)
                    s_0 = s
                    s_1 = ((v * c_1) + random_w)
                else:
                    assert False, "error mu"
                
                c_array.append((c_0, c_1))
                s_array.append((s_0, s_1))
            
            vec_c_array.append(c_array)
            vec_s_array.append(s_array)
        return vec_c_array, vec_s_array
    
    def AKLin1(self):
        print('AKLin1')
        q, g, h = self.pub_key
        vec_b_lin = []
        vec_c_lin = []
        vec_s_lin = []
        vec_random_w_lin = []
        for category, y_array in zip(CATEGORIES, vec_y_array):
            random_w = number.getRandomRange(1, q-1)
            if category != self.data:
                c_p = number.getRandomRange(1, q-1)
                vec_c_lin.append(c_p)
                s_p = number.getRandomRange(1, q-1)
                vec_s_lin.append(s_p)
                g_p_inv = pow(pow(g, (self.n // 2), q), -1, q)
                commitment = 1
                for y in y_array:
                    commitment = commitment * y % q
                deno = pow(commitment * g_p_inv % q, c_p, q)
                b_p = pow(h, s_p, q) * pow(deno, -1, q) % q
                b_q = pow(h, random_w, q)
            else:
                c_q = number.getRandomRange(1, q-1)
                vec_c_lin.append(c_q)
                s_q = number.getRandomRange(1, q-1)
                vec_s_lin.append(s_q)
                g_q_inv = pow(pow(g, (self.l), q), -1, q)
                commitment = 1
                for y in y_array:
                    commitment = commitment * y % q
                deno = pow(commitment * g_q_inv % q, c_q, q)
                b_q = pow(h, s_q, q) * pow(deno, -1, q) % q
                b_p = pow(h, random_w, q)

            vec_b_lin.append((b_p, b_q))
            vec_random_w_lin.append(random_w)

        self.vec_c_lin = vec_c_lin
        self.vec_s_lin = vec_s_lin
        self.vec_b_lin = vec_b_lin
        self.vec_random_w_lin = vec_random_w_lin
        return vec_b_lin

    def AKLin3(self, vec_ak_enc_bool_x_lin):
        print('AKLin3')
        q, g, h = self.pub_key
        vec_c_lin = []
        vec_s_lin = []
        for category, c_lin, s_lin, v_array, random_w_lin, ak_enc_bool_x_lin in zip(
            CATEGORIES,
            self.vec_c_lin,
            self.vec_s_lin,
            self.vec_v_array,
            self.vec_random_w_lin,
            vec_ak_enc_bool_x_lin
        ):
            if category == self.data:
                vec_c_lin.append((ak_enc_bool_x_lin - c_lin, c_lin))
                v_sum = 0
                for v in v_array:
                    v_sum += v
                vec_s_lin.append((v_sum * (ak_enc_bool_x_lin - c_lin) + random_w_lin, s_lin))
            else:
                vec_c_lin.append((c_lin, ak_enc_bool_x_lin - c_lin))
                v_sum = 0
                for v in v_array:
                    v_sum += v
                vec_s_lin.append((s_lin, v_sum * (ak_enc_bool_x_lin - c_lin) + random_w_lin))

        return vec_c_lin, vec_s_lin
    
class Verifier:
    def __init__(self, d, n, l):
        self.d, self.n, self.l = d, n, l
    
    def setup(self, security):
        print('interviewr setup')
        q = number.getPrime(2 * security, Random.new().read)        
        g = number.getRandomRange(1, q-1)
        h = number.getRandomRange(1, q-1)
        
        self.q = q
        self.g = g
        self.h = h
        
        self.sigma_array = np.random.randint(1, self.n+1, self.d).tolist()
        
        print("sigma: ", self.sigma_array)
        
        self.pub_key = (q, g, h)
        
    def step1(self):
        print('step1')
        a_array = [number.getRandomRange(1, self.q-1) for _ in range(self.d)]
        b_array = [number.getRandomRange(1, self.q-1) for _ in range(self.d)]
        self.a_array = a_array
        self.b_array = b_array
        
        g_a_array = [pow(self.g, a, self.q) for a in a_array]
        g_b_array = [pow(self.g, b, self.q) for b in b_array]
        g_ab_array = [pow(self.g, a * b - sigma + 1, self.q) for sigma, a, b in zip(self.sigma_array, a_array, b_array)]

        return g_a_array, g_b_array, g_ab_array
        
    def step3(self, vec_w_array, vec_y_array):
        print('step3')
        result_array = []
        for b, sigma, w_array, y_array in zip(self.b_array, self.sigma_array, vec_w_array, vec_y_array):
            v_sigma = pow(w_array[sigma-1], b, self.q)
            g_mu_sigma = y_array[sigma-1] * pow(pow(self.h, v_sigma, self.q), -1, self.q) % self.q
            for bit in [0, 1]:
                if pow(self.g, bit, self.q) == g_mu_sigma:
                    result_array.append(bit)
        assert len(result_array) == self.d, "invalid verification"
                    
        print("secret output: ", result_array)
        return result_array
    
    def AKEncBool2(self, n, d):
        print('AKEncBool2')
        self.vec_ak_enc_bool_x_array = []
        for _ in range(d):
            self.vec_ak_enc_bool_x_array.append(
                [number.getRandomRange(1, self.q-1) for _ in range(n)]
            )
        return self.vec_ak_enc_bool_x_array

    def AKEncBool4(self, vec_c_array, vec_s_array, vec_b_array, vec_y_array):
        print('AKEncBool4')
        print("####### AKEncBool verification #######")
        for s_array,c_array,b_array,y_array,ak_enc_bool_x_array in zip(
            vec_s_array, vec_c_array, vec_b_array, vec_y_array, self.vec_ak_enc_bool_x_array
        ):
            for s,c,b,y,x in zip(s_array, c_array, b_array, y_array, ak_enc_bool_x_array):
                for i in [0, 1]:
                    if pow(self.h, s[i], self.q) != b[i] * pow(y * pow(pow(self.g, i, self.q), -1, self.q) % self.q, c[i], self.q) % self.q:
                        print("AKEncBool False1.")
                        return False
                if x != sum(c):
                    print("AKEncBool False2.")
                    return False
        print("AKEncBool OK.")
        return True 
    
    def AKLin2(self, d):
        print('AKLin2')
        self.vec_ak_enc_bool_x_lin = []
        for i in range(d):
            self.vec_ak_enc_bool_x_lin.append(number.getRandomRange(1, self.q-1))
        return self.vec_ak_enc_bool_x_lin
    
    def AKLin4(self, vec_c_lin, vec_s_lin, vec_b_lin, vec_y_array):
        print('AKLin4')
        print("####### AKLin verification #######")
        for category, c_lin, s_lin, b_lin, y_array, ak_enc_bool_x_lin in zip(
            CATEGORIES, vec_c_lin, vec_s_lin, vec_b_lin, vec_y_array, self.vec_ak_enc_bool_x_lin
        ):
            commitment = 1
            for y in y_array:
                commitment = commitment * y % self.q
            for c, s, b, summation in zip(c_lin, s_lin, b_lin, [self.n // 2, self.l]):
                if pow(self.h, s, self.q) != b * pow(commitment * pow(pow(self.g, summation, self.q), -1, self.q) % self.q, c, self.q) % self.q:
                    print("AKEncBool False1.")
                    return False
            if ak_enc_bool_x_lin != sum(c_lin):
                print("AKEncBool False2.")
                return False
        print("AKEncBool OK.")
        return True 

In [3]:
CATEGORIES = list(range(0,10))

epsilon = 1.0
secret_input = 2
width = 100
d, n, l = buildParams(epsilon, width)

sender = Prover(secret_input, d, n, l)
sender.setup()

receiver = Verifier(d, n, l)
receiver.setup(security=80)
pub_key = receiver.pub_key

sender.setPubKey(pub_key)

g_a_array, g_b_array, g_ab_array = receiver.step1()
vec_w_array, vec_y_array = sender.step2(g_a_array, g_b_array, g_ab_array)
receiver.step3(vec_w_array, vec_y_array)

vec_b_array = sender.AKEncBool1()
vec_ak_enc_bool_x_array = receiver.AKEncBool2(n, d)
vec_c_array, vec_s_array = sender.AKEncBool3(vec_ak_enc_bool_x_array)
receiver.AKEncBool4(vec_c_array, vec_s_array, vec_b_array, vec_y_array)

vec_b_lin = sender.AKLin1()
vec_ak_enc_bool_x_lin = receiver.AKLin2(d)
vec_c_lin, vec_s_lin = sender.AKLin3(vec_ak_enc_bool_x_lin)
receiver.AKLin4(vec_c_lin, vec_s_lin, vec_b_lin, vec_y_array)

original q= 0.2689414213699951
approximate q= 0.27
n: 100 l: 26 d: 10
receiver setup
secret input:  2
interviewr setup
sigma:  [25, 24, 19, 19, 87, 87, 77, 21, 64, 56]
step1
step2
step3
secret output:  [1, 0, 0, 0, 0, 1, 1, 1, 0, 0]
AKEncBool1
AKEncBool2
AKEncBool3
AKEncBool4
####### AKEncBool verification #######
AKEncBool OK.
AKLin1
AKLin2
AKLin3
AKLin4
####### AKLin verification #######
AKEncBool OK.


True